In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import scipy
from ast import literal_eval
import json
import tqdm

In [4]:
rng = np.random.default_rng(2343)

## Code: truncation functions

In [5]:
def B(x1_OB, x2_OB, x1_OG, x2_OG, beta):
    '''Compute (1 / n) sum j = 1 to n of P(X1_OB, X2_OB | X1_OG, X2_OG, Beta_j): product of binomials.'''
    # currently just work with float beta

    return scipy.stats.binom.pmf(x1_OB, x1_OG, beta) * scipy.stats.binom.pmf(x2_OB, x2_OG, beta)

In [6]:
def BM(x_OB, x_OG, beta):
    '''Compute (1 / n) sum j = 1 to n of P(X_OB | X_OG, Beta_j): binomial prob.'''
    # currently just work with float beta

    return scipy.stats.binom.pmf(x_OB, x_OG, beta)

In [7]:
def findTrunc_old(x1_OB, x2_OB, beta, thresh_OG):
    '''
    Compute box truncation around states (x1_OG, x2_OG) which have
    B(x1_OB, x2_OB, x1_OG, x2_OG, beta) >= thresh_OG

    returns: min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG
    '''

    trunc_start = False
    trunc_end = False
    min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG = np.inf, 0, np.inf, 0
    diag = 0
    while (not trunc_start) or (not trunc_end):

        # start at top of grid
        x1_OG = x1_OB
        x2_OG = x2_OB + diag

        # flag if at least one coeff > thresh in diagonal
        trunc_diag = False

        # compute coeffs along diagonal
        while x2_OG >= x2_OB:

            # compute coeff
            coeff = B(x1_OB, x2_OB, x1_OG, x2_OG, beta)

            # above thresh
            if coeff >= thresh_OG:

                # update truncations
                if x1_OG < min_x1_OG:
                    min_x1_OG = x1_OG
                if x2_OG < min_x2_OG:
                    min_x2_OG = x2_OG
                if x1_OG > max_x1_OG:
                    max_x1_OG = x1_OG
                if x2_OG > max_x2_OG:
                    max_x2_OG = x2_OG

                # at least one coeff > thresh (overall)
                trunc_start = True

                # at least one coeff > thresh (in diag)
                trunc_diag = True

            # move down diagonal
            x2_OG -= 1
            x1_OG += 1

        # if NO coeff > thresh (in diag) AND at least one coeff > thresh (overall)
        if (not trunc_diag) and trunc_start:

            # end
            trunc_end = True

        # increment diagonal
        diag += 1

    return min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG

In [8]:
def findTrunc(x1_OB, x2_OB, beta, thresh_OG):
    '''
    Compute box truncation around states (x1_OG, x2_OG) which have
    B(x1_OB, x2_OB, x1_OG, x2_OG, beta) >= thresh_OG

    returns: m_OG, M_OG, n_OG, N_OG
    '''

    trunc_start = False
    trunc_end = False
    m_OG, M_OG, n_OG, N_OG = np.inf, 0, np.inf, 0
    diag = 0
    while (not trunc_start) or (not trunc_end):

        # flag if at least one coeff > thresh in diagonal
        trunc_diag = False

        # diagonal from upper right to lower left
        x1_OG_diag = np.array([x1_OB + i for i in range(diag + 1)])
        x2_OG_diag = np.array([x2_OB + diag - i for i in range(diag + 1)])

        # compute coeffs
        coeffs = scipy.stats.binom.pmf(x1_OB, x1_OG_diag, beta) * scipy.stats.binom.pmf(x2_OB, x2_OG_diag, beta)

        # find where above threshold
        idxs = np.argwhere(coeffs > 10**-6).reshape(-1)

        # if any values above threshold
        if idxs.size > 0:

            # at least one coeff > thresh (overall)
            trunc_start = True

            # at least one coeff > thresh (in diag)
            trunc_diag = True

            # find states above threshold
            x1_states = x1_OG_diag[idxs]
            x2_states = x2_OG_diag[idxs]

            # find boundaries
            min_x1 = min(x1_states)
            min_x2 = min(x2_states)
            max_x1 = max(x1_states)
            max_x2 = max(x2_states)

            # update truncations
            if min_x1 < m_OG:
                m_OG = min_x1
            if min_x2 < n_OG:
                n_OG = min_x2
            if max_x1 > M_OG:
                M_OG = max_x1
            if max_x2 > N_OG:
                N_OG = max_x2

        # if NO coeff > thresh (in diag) AND at least one coeff > thresh (overall)
        if (not trunc_diag) and trunc_start:

            # end
            trunc_end = True

        # increment diagonal
        diag += 1

    return int(m_OG), int(M_OG), int(n_OG), int(N_OG)

In [9]:
def findTruncM(x_OB, beta, threshM_OG):
    '''
    Compute interval truncation of states x_OG which have
    B(x_OB, x_OG, beta) >= threshM_OG
    
    returns: minM_OG, maxM_OG
    '''

    # start at first non-zero coefficient
    x_OG = x_OB
    coeff = BM(x_OB, x_OG, beta)

    # if not above threshold: increment until above
    while coeff < threshM_OG:

        # increment
        x_OG += 1

        # compute coeff
        coeff = BM(x_OB, x_OG, beta)

    # store first state coeff >= thresh
    minM_OG = x_OG

    # increment until below threshold
    while coeff >= threshM_OG:

        # increment
        x_OG += 1

        # compute coeff
        coeff = BM(x_OB, x_OG, beta)

    # store last state with coeff >= thresh (INCLUSIVE BOUND)
    maxM_OG = x_OG - 1

    return minM_OG, maxM_OG

In [10]:
def preComputeTruncation(M, beta, thresh_OG):
    '''
    Compute dict of original truncations

    M: max state of observed pairs that truncations are computed for
    beta: capture efficiency vector
    thresh_OG: threshold for trunction
    '''
    # store in dictionary (lookup table)
    truncations = {}

    # for each pair of observed counts
    for x1_OB in tqdm.tqdm(range(M + 1)):
        for x2_OB in range(x1_OB + 1):

            # compute truncation bounds
            min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG = findTrunc(x1_OB, x2_OB, beta, thresh_OG)

            # store
            truncations[f'({x1_OB}, {x2_OB})'] = (min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG)

            # store symmetric version
            truncations[f'({x2_OB}, {x1_OB})'] = (min_x2_OG, max_x2_OG, min_x1_OG, max_x1_OG)

    return truncations

In [11]:
def preComputeTruncationM(M, beta, threshM_OG):
    '''
    Compute dict of original truncations

    M: max observed state that truncations are computed for
    beta: capture efficiency vector
    threshM_OG: threshold for trunction
    '''
    # store in dictionary (lookup table)
    truncations = {}

    # for each pair of observed counts
    for x_OB in tqdm.tqdm(range(max)):

            # compute truncation bounds
            minM_OG, maxM_OG = findTruncM(x_OB, beta, threshM_OG)

            # store
            truncations[f'{x_OB}'] = (minM_OG, maxM_OG)

    return truncations

# Perfect information leads to perfect output?

Many of the challenges of our inference arise when dealing with the uncertainity and measurement error present in the observed data, and how to account for that when estimating values of distributions for use in optimization.

However, an equally important part is the optimization step, and the question: If we have the exact distribution values, does the optimzation produce exact bounds on the true parameter values. In other words: given perfect information can we obtain perfect output?

Seeing how close the results in this case are to perfect can tell us how important the bootstrap, and optimization parts of inference are.

## Computing perfect information

To test this we need to compute the exact stationary distribution values

To simplify calculations we focus on the case of no interaction $k_{reg} = 0$ and no observation error $\beta = 100\%$

In this case the stationary distribution of each count is a poisson distribution with parameter equal to transcription rate / degradation rate, and the joint stationary distribution factorises into a product of marginals

In [48]:
params = {
    'k_tx_1': 1,
    'k_tx_2': 1,
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': 0
}

min_x1_OB, max_x1_OB = 0, 3
min_x2_OB, max_x2_OB = 0, 3

bounds = {}
bounds['x1'] = scipy.stats.poisson.pmf(np.array([x1 for x1 in range(min_x1_OB, max_x1_OB + 1)]), params['k_tx_1'] / params['k_deg_1'])
bounds['x2'] = scipy.stats.poisson.pmf(np.array([x2 for x2 in range(min_x2_OB, max_x2_OB + 1)]), params['k_tx_2'] / params['k_deg_2'])
bounds['joint'] = bounds['x1'].reshape(-1, 1) @ bounds['x2'].reshape(1, -1)
bounds['min_x1_OB'] = min_x1_OB
bounds['max_x1_OB'] = max_x1_OB
bounds['min_x2_OB'] = min_x2_OB
bounds['max_x2_OB'] = max_x2_OB

Experiment with including observation error, i.e. capture efficiency of $\beta < 100\% $:

$$ X^{OB} \vert X^{OG}, \beta \sim \text{Bin}(X^{OG}, \beta) $$

$$ X^{OG} \sim \text{Poi}\left(\frac{k_{tx}}{k_{deg}}\right) $$

$$ \implies X^{OB} \sim \text{Poi}\left(\beta \frac{k_{tx}}{k_{deg}}\right)

## Code: distribution computation

In [49]:
def computeDist(params, min_x1_OB, max_x1_OB, min_x2_OB, max_x2_OB, beta=1.0, delta=0.0):
    '''Compute exact values of marginal and joint stationary distributions'''

    bounds = {}

    x1_values = scipy.stats.poisson.pmf(np.array([x1 for x1 in range(min_x1_OB, max_x1_OB + 1)]), beta * params['k_tx_1'] / params['k_deg_1'])
    x2_values = scipy.stats.poisson.pmf(np.array([x2 for x2 in range(min_x2_OB, max_x2_OB + 1)]), beta * params['k_tx_2'] / params['k_deg_2'])
    joint_values = x1_values.reshape(-1, 1) @ x2_values.reshape(1, -1)

    bounds['x1'] = np.stack([x1_values - delta, x1_values + delta], axis=0)
    bounds['x2'] = np.stack([x2_values - delta, x2_values + delta], axis=0)
    bounds['joint'] = np.stack([joint_values - delta, joint_values + delta], axis=0)
    bounds['min_x1_OB'] = min_x1_OB
    bounds['max_x1_OB'] = max_x1_OB
    bounds['min_x2_OB'] = min_x2_OB
    bounds['max_x2_OB'] = max_x2_OB

    return bounds

## Code: marginal optimization

In [12]:
def optimization_hyp_single(bounds, beta, truncationsM, gene, K=100, silent=True,
                     print_solution=True, print_truncation=True, threshM_OG=10**-6,
                     time_limit=300):

    # create model
    md = gp.Model('birth-death-regulation-capture-efficiency-hyp')

    # set options
    if silent:
        md.Params.LogToConsole = 0

    # set time limit: 5 minute default
    md.Params.TimeLimit = time_limit

    # State space truncations

    # marginal observed truncation
    min_OB = bounds[f'min_x{gene}_OB']
    max_OB = bounds[f'max_x{gene}_OB']

    # original truncations: find largest original states needed (to define variables)
    overall_min_OG, overall_max_OG = np.inf, 0

    # for each marginal state used
    for x_OB in range(min_OB, max_OB + 1):

        try:
            # lookup original truncation
            min_OG, max_OG = truncationsM[f'{x_OB}']

        except KeyError:
            # compute if not available
            min_OG, max_OG = findTruncM(x_OB, beta, threshM_OG)

            # store
            truncationsM[f'{x_OB}'] = (min_OG, max_OG)

        # update overall min and max
        if max_OG > overall_max_OG:
            overall_max_OG = max_OG
        if min_OG < overall_min_OG:
            overall_min_OG = min_OG
    
    if print_truncation:
        print(f"Observed counts: [{min_OB}, {max_OB}]")
        print(f"Original counts: [{overall_min_OG}, {overall_max_OG}]")

    # variables

    # marginal stationary distributions: original counts (size = largest original state + 1)
    p = md.addMVar(shape=(overall_max_OG + 1), vtype=GRB.CONTINUOUS, name="p", lb=0, ub=1)

    '''aggressive presolve to hopefully ensure this'''
    md.Params.Presolve = 2

    # reaction rate constants
    rate_names = ['k_tx', 'k_deg']
    rates = md.addVars(rate_names, vtype=GRB.CONTINUOUS, lb=0, ub=K, name=rate_names)

    # constraints

    # fix k_deg_1 = 1 for identifiability
    md.addConstr(rates['k_deg'] == 1)

    # distributional constraints
    md.addConstr(p.sum() <= 1, name="Distribution")
    
    # marginal stationary distribution bounds: for each observed count
    for x_OB in range(min_OB, max_OB + 1):

        # original truncation: lookup from pre-computed dict
        min_OG, max_OG = truncationsM[f'{x_OB}']

        # sum over truncation range (INCLUSIVE)
        sum_expr = gp.quicksum([BM(x_OB, x_OG, beta) * p[x_OG] for x_OG in range(min_OG, max_OG + 1)])

        md.addConstr(sum_expr >= bounds[f'x{gene}'][0, x_OB], name=f"B marginal lb {x_OB}")
        md.addConstr(sum_expr <= bounds[f'x{gene}'][1, x_OB], name=f"B marginal ub {x_OB}")

    # CME
    for x_OG in range(overall_max_OG):
        if x_OG == 0:
            x_zero = 0
        else:
            x_zero = 1

        md.addConstr(
            rates['k_tx'] * x_zero * p[x_OG - 1] + \
            rates['k_deg'] * (x_OG + 1) * p[x_OG + 1] - \
            (rates['k_tx'] + rates['k_deg'] * x_OG) * p[x_OG] == 0,
            name=f"Marginal CME {x_OG}"
        )

    # status of optimization
    status_codes = {1: 'LOADED',
                    2: 'OPTIMAL',
                    3: 'INFEASIBLE',
                    4: 'INF_OR_UNBD',
                    5: 'UNBOUNDED',
                    6: 'CUTOFF',
                    7: 'ITERATION_LIMIT',
                    8: 'NODE_LIMIT',
                    9: 'TIME_LIMIT',
                    10: 'SOLUTION_LIMIT',
                    11: 'INTERRUPTED',
                    12: 'NUMERIC',
                    13: 'SUBOPTIMAL',
                    14: 'INPROGRESS',
                    15: 'USER_OBJ_LIMIT'}

    # solution dict
    solution = {
        'status': None,
        'k_tx': None,
        'k_deg': 1,
        'min_time': None,
        'max_time': None
    }

    # optimize

    # minimize
    md.setObjective(rates['k_tx'], GRB.MINIMIZE)
    try:
        md.optimize()
        min_val = md.ObjVal
        min_status = status_codes[md.status]
        min_time = md.Runtime
    except:
        min_val = None
        min_status = status_codes[md.status]
        min_time = md.Runtime

    # maximize
    md.setObjective(rates['k_tx'], GRB.MAXIMIZE)
    try:
        md.optimize()
        max_val = md.ObjVal
        max_status = status_codes[md.status]
        max_time = md.Runtime
    except:
        max_val = None
        max_status = status_codes[md.status]
        max_time = md.Runtime

    # store
    solution['k_tx'] = [min_val, min_status, max_val, max_status]
    solution['min_time'] = min_time
    solution['max_time'] = max_time

    if print_solution:
        print(f"k_tx lower bound: {solution['k_tx'][0]}, status: {solution['k_tx'][1]}, time: {solution['min_time']}")
        print(f"k_tx upper bound: {solution['k_tx'][2]}, status: {solution['k_tx'][3]}, time: {solution['max_time']}")
        print(f"k_deg: 1")

    # save runtime
    solution['time'] = md.Runtime

    return solution

## Code: hypothesis optimization (with constraint options)

In [13]:
def optimization_hyp(params, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=True, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=300, settings={}):

    # create model
    md = gp.Model('birth-death-regulation-capture-efficiency-hyp')

    # set options
    if silent:
        md.Params.LogToConsole = 0

    # set time limit: 5 minute default
    md.Params.TimeLimit = time_limit

    # State space truncations

    # observed truncations: computed during bootstrap
    min_x1_OB = bounds['min_x1_OB']
    max_x1_OB = bounds['max_x1_OB']
    min_x2_OB = bounds['min_x2_OB']
    max_x2_OB = bounds['max_x2_OB']

    # marginal observed truncations: use same as joint for now
    minM_x1_OB = bounds['min_x1_OB']
    maxM_x1_OB = bounds['max_x1_OB']
    minM_x2_OB = bounds['min_x2_OB']
    maxM_x2_OB = bounds['max_x2_OB']

    # original truncations: find largest original states needed (to define variables)
    overall_min_x1_OG, overall_max_x1_OG, overall_min_x2_OG, overall_max_x2_OG = np.inf, 0, np.inf, 0

    # for each pair of observed states used
    for x1_OB in range(min_x1_OB, max_x1_OB + 1):
        for x2_OB in range(min_x2_OB, max_x2_OB + 1):

            try:
                # lookup original truncation
                min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG = truncations[f'({x1_OB}, {x2_OB})']

            except KeyError:
                # compute if not available
                min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG = findTrunc(x1_OB, x2_OB, beta, thresh_OG)

                # store
                truncations[f'({x1_OB}, {x2_OB})'] = (min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG)

            # if larger than current maximum states: update
            if max_x1_OG > overall_max_x1_OG:
                overall_max_x1_OG = max_x1_OG
            if max_x2_OG > overall_max_x2_OG:
                overall_max_x2_OG = max_x2_OG

            # if smaller than current minimum states: update
            if min_x1_OG < overall_min_x1_OG:
                overall_min_x1_OG = min_x2_OG
            if min_x2_OG < overall_min_x2_OG:
                overall_min_x2_OG = min_x2_OG

    # for each x1 marginal state used
    for x1_OB in range(minM_x1_OB, maxM_x1_OB + 1):

        try:
            # lookup original truncation
            minM_x1_OG, maxM_x1_OG = truncationsM[f'{x1_OB}']

        except KeyError:
            # compute if not available
            minM_x1_OG, maxM_x1_OG = findTruncM(x1_OB, beta, threshM_OG)

            # store
            truncationsM[f'{x1_OB}'] = (minM_x1_OG, maxM_x1_OG)

        # update overall min and max
        if maxM_x1_OG > overall_max_x1_OG:
            overall_max_x1_OG = maxM_x1_OG
        if minM_x1_OG < overall_min_x1_OG:
            overall_min_x1_OG = minM_x1_OG

    # for each x2 marginal state used
    for x2_OB in range(minM_x2_OB, maxM_x2_OB + 1):

        try:
            # lookup original truncation
            minM_x2_OG, maxM_x2_OG = truncationsM[f'{x2_OB}']

        except KeyError:
            # compute if not available
            minM_x2_OG, maxM_x2_OG = findTruncM(x2_OB, beta, threshM_OG)

            # store
            truncationsM[f'{x2_OB}'] = (minM_x2_OG, maxM_x2_OG)

        # update overall min and max
        if maxM_x2_OG > overall_max_x2_OG:
            overall_max_x2_OG = maxM_x2_OG
        if minM_x2_OG < overall_min_x2_OG:
            overall_min_x2_OG = minM_x2_OG
    
    if print_truncation:
        print(f"Observed counts: [{min_x1_OB}, {max_x1_OB}] x [{min_x2_OB}, {max_x2_OB}]")
        print(f"Original counts: [{overall_min_x1_OG}, {overall_max_x1_OG}] x [{overall_min_x2_OG}, {overall_max_x2_OG}]")

    # variables

    # marginal stationary distributions: original counts (size = largest original state + 1)
    p1 = md.addMVar(shape=(overall_max_x1_OG + 1), vtype=GRB.CONTINUOUS, name="p1", lb=0, ub=1)
    p2 = md.addMVar(shape=(overall_max_x2_OG + 1), vtype=GRB.CONTINUOUS, name="p2", lb=0, ub=1)

    # dummy joint variable to avoid triple products (as not supported by GUROBI): should be removed by presolve
    p_dummy = md.addMVar(shape=(overall_max_x1_OG + 1, overall_max_x2_OG + 1), vtype=GRB.CONTINUOUS, name="p_dummy", lb=0, ub=1)

    '''aggressive presolve to hopefully ensure this'''
    md.Params.Presolve = 2

    # reaction rate constants
    rate_names = ['k_tx_1', 'k_tx_2', 'k_deg_1', 'k_deg_2']
    rates = md.addVars(rate_names, vtype=GRB.CONTINUOUS, lb=0, ub=K, name=rate_names)

    # constraints

    # set fixed reaction rates
    for name, value in params.items():
        if value == "v" or value == "vo":
            pass
        else:
            md.addConstr(rates[name] == value)

    # distributional constraints
    md.addConstr(p1.sum() <= 1, name="Distribution x1")
    md.addConstr(p2.sum() <= 1, name="Distribution x2")

    if settings['bivariateB']:

        # stationary distribution bounds: for each observed count pair
        for x1_OB in range(min_x1_OB, max_x1_OB + 1):
            for x2_OB in range(min_x2_OB, max_x2_OB + 1):
                
                # original truncation: lookup from pre-computed dict
                min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG = truncations[f'({x1_OB}, {x2_OB})']
                
                # sum over truncation range (INCLUSIVE): drop terms with coefficients < thresh
                sum_expr = gp.quicksum([
                    B(x1_OB, x2_OB, x1_OG, x2_OG, beta) * p1[x1_OG] * p2[x2_OG]
                    for x1_OG in range(min_x1_OG, max_x1_OG + 1)
                    for x2_OG in range(min_x2_OG, max_x2_OG + 1)
                    if B(x1_OB, x2_OB, x1_OG, x2_OG, beta) >= thresh_OG
                ])
                
                md.addConstr(sum_expr >= bounds['joint'][0, x1_OB, x2_OB], name=f"B lb {x1_OB}, {x2_OB}")
                md.addConstr(sum_expr <= bounds['joint'][1, x1_OB, x2_OB], name=f"B ub {x1_OB}, {x2_OB}")
    
    if settings['univariateB']:

        # marginal stationary distribution bounds: for each observed count
        for x1_OB in range(minM_x1_OB, maxM_x1_OB + 1):

            # original truncation: lookup from pre-computed dict
            minM_x1_OG, maxM_x1_OG = truncationsM[f'{x1_OB}']

            # sum over truncation range (INCLUSIVE)
            sum_expr = gp.quicksum([BM(x1_OB, x1_OG, beta) * p1[x1_OG] for x1_OG in range(minM_x1_OG, maxM_x1_OG + 1)])

            md.addConstr(sum_expr >= bounds['x1'][0, x1_OB], name=f"B marginal lb {x1_OB}")
            md.addConstr(sum_expr <= bounds['x1'][1, x1_OB], name=f"B marginal ub {x1_OB}")

        for x2_OB in range(minM_x2_OB, maxM_x2_OB + 1):

            # original truncation: lookup from pre-computed dict
            minM_x2_OG, maxM_x2_OG = truncationsM[f'{x2_OB}']

            # sum over truncation range (INCLUSIVE)
            sum_expr = gp.quicksum([BM(x2_OB, x2_OG, beta) * p2[x2_OG] for x2_OG in range(minM_x2_OG, maxM_x2_OG + 1)])

            md.addConstr(sum_expr >= bounds['x2'][0, x2_OB], name=f"B marginal lb {x2_OB}")
            md.addConstr(sum_expr <= bounds['x2'][1, x2_OB], name=f"B marginal ub {x2_OB}")

    if settings['bivariateCME']:

        # equate dummy joint variable to product of marginals: all original states
        for x1_OG in range(overall_max_x1_OG + 1):
            for x2_OG in range(overall_max_x2_OG + 1):

                md.addConstr(p_dummy[x1_OG, x2_OG] == p1[x1_OG] * p2[x2_OG], name=f"Dummy joint definition {x1_OG}, {x2_OG}")

        # CME: use dummy joint variable to avoid triple products: k_[] * p1[] * p2[]
        for x1_OG in range(overall_max_x1_OG):
            for x2_OG in range(overall_max_x2_OG):

                # remove terms when x's = 0 as not present in equation
                if x1_OG == 0:
                    x1_zero = 0
                else:
                    x1_zero = 1
                if x2_OG == 0:
                    x2_zero = 0
                else:
                    x2_zero = 1

                md.addConstr(
                    rates['k_tx_1'] * x1_zero * p_dummy[x1_OG - 1, x2_OG] + \
                    rates['k_tx_2'] * x2_zero * p_dummy[x1_OG, x2_OG - 1] + \
                    rates['k_deg_1'] * (x1_OG + 1) * p_dummy[x1_OG + 1, x2_OG] + \
                    rates['k_deg_2'] * (x2_OG + 1) * p_dummy[x1_OG, x2_OG + 1] - \
                    (rates['k_tx_1'] + rates['k_tx_2'] + \
                    rates['k_deg_1'] * x1_OG + rates['k_deg_2'] * x2_OG) * p_dummy[x1_OG, x2_OG] == 0,
                    name=f"CME {x1_OG}, {x2_OG}"
                    )
    
    if settings['univariateCME']:

        # CME for x1
        for x1_OG in range(overall_max_x1_OG):
            if x1_OG == 0:
                x1_zero = 0
            else:
                x1_zero = 1

            md.addConstr(
                rates['k_tx_1'] * x1_zero * p1[x1_OG - 1] + \
                rates['k_deg_1'] * (x1_OG + 1) * p1[x1_OG + 1] - \
                (rates['k_tx_1'] + rates['k_deg_1'] * x1_OG) * p1[x1_OG] == 0,
                name=f"Marginal CME x1 {x1_OG}"
            )

        # CME for x2
        for x2_OG in range(overall_max_x2_OG):
            if x2_OG == 0:
                x2_zero = 0
            else:
                x2_zero = 1

            md.addConstr(
                rates['k_tx_2'] * x2_zero * p2[x2_OG - 1] + \
                rates['k_deg_2'] * (x2_OG + 1) * p2[x2_OG + 1] - \
                (rates['k_tx_2'] + rates['k_deg_2'] * x2_OG) * p2[x2_OG] == 0,
                name=f"Marginal CME x2 {x2_OG}"
            )

    # status of optimization
    status_codes = {1: 'LOADED',
                    2: 'OPTIMAL',
                    3: 'INFEASIBLE',
                    4: 'INF_OR_UNBD',
                    5: 'UNBOUNDED',
                    6: 'CUTOFF',
                    7: 'ITERATION_LIMIT',
                    8: 'NODE_LIMIT',
                    9: 'TIME_LIMIT',
                    10: 'SOLUTION_LIMIT',
                    11: 'INTERRUPTED',
                    12: 'NUMERIC',
                    13: 'SUBOPTIMAL',
                    14: 'INPROGRESS',
                    15: 'USER_OBJ_LIMIT'}

    # solution dict
    solution = {}

    # optimize
    for name, value in params.items():

        # variable
        if value == "vo":

            # minimize
            md.setObjective(rates[name], GRB.MINIMIZE)
            try:
                md.optimize()
                min_val = md.ObjVal
                min_status = status_codes[md.status]
                min_time = md.Runtime
            except:
                min_val = None
                min_status = status_codes[md.status]
                min_time = md.Runtime

            # maximize
            md.setObjective(rates[name], GRB.MAXIMIZE)
            try:
                md.optimize()
                max_val = md.ObjVal
                max_status = status_codes[md.status]
                max_time = md.Runtime
            except:
                max_val = None
                max_status = status_codes[md.status]
                max_time = md.Runtime

            # store
            solution[name] = [min_val, min_status, min_time, max_val, max_status, max_time]
    
        # constant
        else:
            solution[name] = value
    '''
    # testing feasibility: simply optimize 0
    md.setObjective(0, GRB.MINIMIZE)

    # set parameter (prevents 'infeasible or unbounded' ambiguity)
    md.Params.DualReductions = 0

    # set solution limit (stop after finding 1 feasible solution)
    md.Params.SolutionLimit = 1

    try:
        md.optimize()
        status_code = md.status
    except:
        status_code = md.status

    # store result
    solution['status'] = status_codes[status_code]
    '''

    # print
    '''
    if print_solution:
        for key, val in solution.items():
            if key == "status":
                print(f"Model is {val}")
            elif val == "v":
                print(f"{key} variable")
            else:
                print(f"{key} = {val}")
    '''

    if print_solution:
        for key, val in params.items():
            if val == "v":
                print(f"{key}: variable")
            elif val == "vo":
                print(f"{key} lower bound: {solution[key][0]}, status: {solution[key][1]}, time: {solution[key][2]}")
                print(f"{key} upper bound: {solution[key][3]}, status: {solution[key][4]}, time: {solution[key][5]}")
            else:
                print(f"{key} = {val}")


    return solution

## Code: standard optimization (no assumption no interaction)

In [14]:
def optimization_min(params, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=True, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=300, settings={}):

    # create model
    md = gp.Model('birth-death-regulation-capture-efficiency-min')

    # set options
    if silent:
        md.Params.LogToConsole = 0

    # set time limit: 5 minute default
    md.Params.TimeLimit = time_limit

    # State space truncations

    # observed truncations: computed during bootstrap
    min_x1_OB = bounds['min_x1_OB']
    max_x1_OB = bounds['max_x1_OB']
    min_x2_OB = bounds['min_x2_OB']
    max_x2_OB = bounds['max_x2_OB']

    # marginal observed truncations: use same as joint for now
    minM_x1_OB = bounds['min_x1_OB']
    maxM_x1_OB = bounds['max_x1_OB']
    minM_x2_OB = bounds['min_x2_OB']
    maxM_x2_OB = bounds['max_x2_OB']

    # original truncations: find largest original states needed (to define variables)
    overall_min_x1_OG, overall_max_x1_OG, overall_min_x2_OG, overall_max_x2_OG = np.inf, 0, np.inf, 0

    # for each pair of observed states used
    for x1_OB in range(min_x1_OB, max_x1_OB + 1):
        for x2_OB in range(min_x2_OB, max_x2_OB + 1):

            try:
                # lookup original truncation
                min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG = truncations[f'({x1_OB}, {x2_OB})']

            except KeyError:
                # compute if not available
                min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG = findTrunc(x1_OB, x2_OB, beta, thresh_OG)

                # store
                truncations[f'({x1_OB}, {x2_OB})'] = (min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG)

            # if larger than current maximum states: update
            if max_x1_OG > overall_max_x1_OG:
                overall_max_x1_OG = max_x1_OG
            if max_x2_OG > overall_max_x2_OG:
                overall_max_x2_OG = max_x2_OG

            # if smaller than current minimum states: update
            if min_x1_OG < overall_min_x1_OG:
                overall_min_x1_OG = min_x2_OG
            if min_x2_OG < overall_min_x2_OG:
                overall_min_x2_OG = min_x2_OG

    # for each x1 marginal state used
    for x1_OB in range(minM_x1_OB, maxM_x1_OB + 1):

        try:
            # lookup original truncation
            minM_x1_OG, maxM_x1_OG = truncationsM[f'{x1_OB}']

        except KeyError:
            # compute if not available
            minM_x1_OG, maxM_x1_OG = findTruncM(x1_OB, beta, threshM_OG)

            # store
            truncationsM[f'{x1_OB}'] = (minM_x1_OG, maxM_x1_OG)

        # update overall min and max
        if maxM_x1_OG > overall_max_x1_OG:
            overall_max_x1_OG = maxM_x1_OG
        if minM_x1_OG < overall_min_x1_OG:
            overall_min_x1_OG = minM_x1_OG

    # for each x2 marginal state used
    for x2_OB in range(minM_x2_OB, maxM_x2_OB + 1):

        try:
            # lookup original truncation
            minM_x2_OG, maxM_x2_OG = truncationsM[f'{x2_OB}']

        except KeyError:
            # compute if not available
            minM_x2_OG, maxM_x2_OG = findTruncM(x2_OB, beta, threshM_OG)

            # store
            truncationsM[f'{x2_OB}'] = (minM_x2_OG, maxM_x2_OG)

        # update overall min and max
        if maxM_x2_OG > overall_max_x2_OG:
            overall_max_x2_OG = maxM_x2_OG
        if minM_x2_OG < overall_min_x2_OG:
            overall_min_x2_OG = minM_x2_OG
    
    if print_truncation:
        print(f"Observed counts: [{min_x1_OB}, {max_x1_OB}] x [{min_x2_OB}, {max_x2_OB}]")
        print(f"Original counts: [{overall_min_x1_OG}, {overall_max_x1_OG}] x [{overall_min_x2_OG}, {overall_max_x2_OG}]")

    # variables

    # joint stationary distribution
    p = md.addMVar(shape=(overall_max_x1_OG + 1, overall_max_x2_OG + 1), vtype=GRB.CONTINUOUS, name="p", lb=0, ub=1)

    # marginal stationary distributions: original counts (size = largest original state + 1)
    p1 = md.addMVar(shape=(overall_max_x1_OG + 1), vtype=GRB.CONTINUOUS, name="p1", lb=0, ub=1)
    p2 = md.addMVar(shape=(overall_max_x2_OG + 1), vtype=GRB.CONTINUOUS, name="p2", lb=0, ub=1)

    '''aggressive presolve to hopefully ensure this'''
    md.Params.Presolve = 2

    # reaction rate constants
    rate_names = ['k_tx_1', 'k_tx_2', 'k_deg_1', 'k_deg_2', 'k_reg']
    rates = md.addVars(rate_names, vtype=GRB.CONTINUOUS, lb=0, ub=K, name=rate_names)

    # constraints

    # set fixed reaction rates
    for name, value in params.items():
        if value == "v" or value == "vo":
            pass
        else:
            md.addConstr(rates[name] == value)

    # distributional constraints
    md.addConstr(p.sum() <= 1, name="Distribution")
    md.addConstr(p1.sum() <= 1, name="Distribution x1")
    md.addConstr(p2.sum() <= 1, name="Distribution x2")

    if settings['bivariateB']:

        # stationary distribution bounds: for each observed count pair
        for x1_OB in range(min_x1_OB, max_x1_OB + 1):
            for x2_OB in range(min_x2_OB, max_x2_OB + 1):
                
                # original truncation: lookup from pre-computed dict
                min_x1_OG, max_x1_OG, min_x2_OG, max_x2_OG = truncations[f'({x1_OB}, {x2_OB})']
                
                # sum over truncation range (INCLUSIVE): drop terms with coefficients < thresh
                sum_expr = gp.quicksum([
                    B(x1_OB, x2_OB, x1_OG, x2_OG, beta) * p[x1_OG, x2_OG]
                    for x1_OG in range(min_x1_OG, max_x1_OG + 1)
                    for x2_OG in range(min_x2_OG, max_x2_OG + 1)
                    if B(x1_OB, x2_OB, x1_OG, x2_OG, beta) >= thresh_OG
                ])
                
                md.addConstr(sum_expr >= bounds['joint'][0, x1_OB, x2_OB], name=f"B lb {x1_OB}, {x2_OB}")
                md.addConstr(sum_expr <= bounds['joint'][1, x1_OB, x2_OB], name=f"B ub {x1_OB}, {x2_OB}")
    
    if settings['univariateB']:

        # frechet constraints: link marginal to joint
        for x1_OG in range(overall_max_x1_OG + 1):
            for x2_OG in range(overall_min_x2_OG + 1):
                md.addConstr(p[x1_OG, x2_OG] >= p1[x1_OG] + p2[x2_OG] - 1, name=f"Frechet lb {x1_OG}, {x2_OG}")
                md.addConstr(p[x1_OG, x2_OG] <= p1[x1_OG], name=f"Frechet ub x1 {x1_OG}, {x2_OG}")
                md.addConstr(p[x1_OG, x2_OG] <= p2[x2_OG], name=f"Frechet ub x2 {x2_OG}, {x2_OG}")

        # marginal stationary distribution bounds: for each observed count
        for x1_OB in range(minM_x1_OB, maxM_x1_OB + 1):

            # original truncation: lookup from pre-computed dict
            minM_x1_OG, maxM_x1_OG = truncationsM[f'{x1_OB}']

            # sum over truncation range (INCLUSIVE)
            sum_expr = gp.quicksum([BM(x1_OB, x1_OG, beta) * p1[x1_OG] for x1_OG in range(minM_x1_OG, maxM_x1_OG + 1)])

            md.addConstr(sum_expr >= bounds['x1'][0, x1_OB], name=f"B marginal lb {x1_OB}")
            md.addConstr(sum_expr <= bounds['x1'][1, x1_OB], name=f"B marginal ub {x1_OB}")

        for x2_OB in range(minM_x2_OB, maxM_x2_OB + 1):

            # original truncation: lookup from pre-computed dict
            minM_x2_OG, maxM_x2_OG = truncationsM[f'{x2_OB}']

            # sum over truncation range (INCLUSIVE)
            sum_expr = gp.quicksum([BM(x2_OB, x2_OG, beta) * p2[x2_OG] for x2_OG in range(minM_x2_OG, maxM_x2_OG + 1)])

            md.addConstr(sum_expr >= bounds['x2'][0, x2_OB], name=f"B marginal lb {x2_OB}")
            md.addConstr(sum_expr <= bounds['x2'][1, x2_OB], name=f"B marginal ub {x2_OB}")

    if settings['bivariateCME']:

        # CME: use dummy joint variable to avoid triple products: k_[] * p1[] * p2[]
        for x1_OG in range(overall_max_x1_OG):
            for x2_OG in range(overall_max_x2_OG):

                # remove terms when x's = 0 as not present in equation
                if x1_OG == 0:
                    x1_zero = 0
                else:
                    x1_zero = 1
                if x2_OG == 0:
                    x2_zero = 0
                else:
                    x2_zero = 1

                md.addConstr(
                        rates['k_tx_1'] * x1_zero * p[x1_OG - 1, x2_OG] + \
                        rates['k_tx_2'] * x2_zero * p[x1_OG, x2_OG - 1] + \
                        rates['k_deg_1'] * (x1_OG + 1) * p[x1_OG + 1, x2_OG] + \
                        rates['k_deg_2'] * (x2_OG + 1) * p[x1_OG, x2_OG + 1] + \
                        rates['k_reg'] * (x1_OG + 1) * (x2_OG + 1) * p[x1_OG + 1, x2_OG + 1] - \
                        (rates['k_tx_1'] + rates['k_tx_2'] + \
                        rates['k_deg_1'] * x1_OG + rates['k_deg_2'] * x2_OG + \
                        rates['k_reg'] * x1_OG * x2_OG) * p[x1_OG, x2_OG] == 0,
                        name=f"Equation {x1_OG}, {x2_OG}"
                        )

    # status of optimization
    status_codes = {1: 'LOADED',
                    2: 'OPTIMAL',
                    3: 'INFEASIBLE',
                    4: 'INF_OR_UNBD',
                    5: 'UNBOUNDED',
                    6: 'CUTOFF',
                    7: 'ITERATION_LIMIT',
                    8: 'NODE_LIMIT',
                    9: 'TIME_LIMIT',
                    10: 'SOLUTION_LIMIT',
                    11: 'INTERRUPTED',
                    12: 'NUMERIC',
                    13: 'SUBOPTIMAL',
                    14: 'INPROGRESS',
                    15: 'USER_OBJ_LIMIT'}

    # solution dict
    solution = {}

    # optimize
    for name, value in params.items():

        # variable
        if value == "vo":

            # minimize
            md.setObjective(rates[name], GRB.MINIMIZE)
            try:
                md.optimize()
                min_val = md.ObjVal
                min_status = status_codes[md.status]
                min_time = md.Runtime
            except:
                min_val = None
                min_status = status_codes[md.status]
                min_time = md.Runtime

            # maximize
            md.setObjective(rates[name], GRB.MAXIMIZE)
            try:
                md.optimize()
                max_val = md.ObjVal
                max_status = status_codes[md.status]
                max_time = md.Runtime
            except:
                max_val = None
                max_status = status_codes[md.status]
                max_time = md.Runtime

            # store
            solution[name] = [min_val, min_status, min_time, max_val, max_status, max_time]
    
        # constant
        else:
            solution[name] = value
    '''
    # testing feasibility: simply optimize 0
    md.setObjective(0, GRB.MINIMIZE)

    # set parameter (prevents 'infeasible or unbounded' ambiguity)
    md.Params.DualReductions = 0

    # set solution limit (stop after finding 1 feasible solution)
    md.Params.SolutionLimit = 1

    try:
        md.optimize()
        status_code = md.status
    except:
        status_code = md.status

    # store result
    solution['status'] = status_codes[status_code]
    '''

    # print
    '''
    if print_solution:
        for key, val in solution.items():
            if key == "status":
                print(f"Model is {val}")
            elif val == "v":
                print(f"{key} variable")
            else:
                print(f"{key} = {val}")
    '''

    if print_solution:
        for key, val in params.items():
            if val == "v":
                print(f"{key}: variable")
            elif val == "vo":
                print(f"{key} lower bound: {solution[key][0]}, status: {solution[key][1]}, time: {solution[key][2]}")
                print(f"{key} upper bound: {solution[key][3]}, status: {solution[key][4]}, time: {solution[key][5]}")
            else:
                print(f"{key} = {val}")


    return solution

## Test: (hyp) marginal optimization, perfect information

In [53]:
# reaction rates: no interaction
params = {
    'k_tx_1': 2,
    'k_tx_2': 2,
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': 0
}

# capture efficiency
beta = 0.5

# truncations (will be trivial i.e. x_OB: (x_OB, x_OB) as 100% capture)
truncationsM = {}

# compute exact values ('bounds')
bounds = computeDist(params, 0, 5, 0, 5, beta=beta, delta=0.0)

In [54]:
solution_x1 = optimization_hyp_single(bounds, beta, truncationsM, gene=1, K=100, silent=True,
                     print_solution=True, print_truncation=False, threshM_OG=10**-6,
                     time_limit=300)

k_tx lower bound: 1.9999997389625588, status: OPTIMAL, time: 0.08800005912780762
k_tx upper bound: 2.0000000029969787, status: OPTIMAL, time: 0.0690000057220459
k_deg: 1


In [55]:
solution_x2 = optimization_hyp_single(bounds, beta, truncationsM, gene=2, K=100, silent=True,
                     print_solution=True, print_truncation=False, threshM_OG=10**-6,
                     time_limit=300)

k_tx lower bound: 1.9999997389625588, status: OPTIMAL, time: 0.03299999237060547
k_tx upper bound: 2.0000000029969787, status: OPTIMAL, time: 0.04900002479553223
k_deg: 1


## Test: (hyp) full optimization, perfect information

In [57]:
# reaction rates: no interaction
params = {
    'k_tx_1': 2,
    'k_tx_2': 2,
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': 0
}

# capture efficiency
beta = 0.5

# truncations (will be trivial i.e. x_OB: (x_OB, x_OB) as 100% capture)
truncations = {}
truncationsM = {}

# compute exact values ('bounds')
bounds = computeDist(params, 0, 5, 0, 5, beta=beta, delta=0.0)

In [58]:
# parameter settings
params_optim = {
    'k_tx_1': "vo",
    'k_tx_2': "vo",
    'k_deg_1': 1,
    'k_deg_2': 1,
}

# constraint settings
settings = {
    'bivariateB': False,
    'univariateB': True,
    'bivariateCME': False,
    'univariateCME': True
}

# optimize
solution = optimization_hyp(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=300, settings=settings)

k_tx_1 lower bound: 1.9999997389580793, status: OPTIMAL, time: 0.03299999237060547
k_tx_1 upper bound: 2.0, status: OPTIMAL, time: 0.0820000171661377
k_tx_2 lower bound: 1.9999999961507093, status: OPTIMAL, time: 0.06800007820129395
k_tx_2 upper bound: 2.0000000029969804, status: OPTIMAL, time: 0.04999995231628418
k_deg_1 = 1
k_deg_2 = 1


In [59]:
# parameter settings
params_optim = {
    'k_tx_1': "vo",
    'k_tx_2': "vo",
    'k_deg_1': 1,
    'k_deg_2': 1,
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
    'univariateCME': True
}

# optimize
solution = optimization_hyp(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=300, settings=settings)

k_tx_1 lower bound: 1.999999910188748, status: OPTIMAL, time: 4.424999952316284
k_tx_1 upper bound: 2.000000032446394, status: OPTIMAL, time: 55.103999853134155
k_tx_2 lower bound: 2.0, status: OPTIMAL, time: 31.330000162124634
k_tx_2 upper bound: 2.000000032446394, status: OPTIMAL, time: 22.717000007629395
k_deg_1 = 1
k_deg_2 = 1


## Test: (min) standard optimization, perfect information

In [60]:
# reaction rates: no interaction
params = {
    'k_tx_1': 2,
    'k_tx_2': 2,
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': 0
}

# capture efficiency
beta = 0.9

# truncations (will be trivial i.e. x_OB: (x_OB, x_OB) as 100% capture)
truncations = {}
truncationsM = {}

# compute exact values ('bounds')
bounds = computeDist(params, 0, 5, 0, 5, beta=beta, delta=10**-6)

### Both degradation rates fixed

In [64]:
# parameter settings
params_optim = {
    'k_tx_1': "vo",
    'k_tx_2': "vo",
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=600, settings=settings)

k_tx_1 lower bound: 1.999959490737254, status: OPTIMAL, time: 15.833999872207642
k_tx_1 upper bound: 2.000003281734897, status: OPTIMAL, time: 12.917999982833862
k_tx_2 lower bound: 1.9999613614708176, status: OPTIMAL, time: 13.36300015449524
k_tx_2 upper bound: 2.00010421614029, status: OPTIMAL, time: 10.036999940872192
k_deg_1 = 1
k_deg_2 = 1
k_reg lower bound: 0.0, status: OPTIMAL, time: 5.228999853134155
k_reg upper bound: 4.431138432359791e-05, status: OPTIMAL, time: 32.77900004386902


### Both degradation rates fixed + no joint B constraints

In [65]:
# parameter settings
params_optim = {
    'k_tx_1': "vo",
    'k_tx_2': "vo",
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': False,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=600, settings=settings)

k_tx_1 lower bound: 0.0, status: OPTIMAL, time: 0.15600013732910156
k_tx_1 upper bound: 100.0, status: OPTIMAL, time: 0.0
k_tx_2 lower bound: 0.0, status: OPTIMAL, time: 0.016000032424926758
k_tx_2 upper bound: 100.0, status: OPTIMAL, time: 0.014999866485595703
k_deg_1 = 1
k_deg_2 = 1
k_reg lower bound: 0.0, status: OPTIMAL, time: 0.016000032424926758
k_reg upper bound: 100.0, status: OPTIMAL, time: 0.016000032424926758


### One degradation rate fixed

In [ ]:
# parameter settings
params_optim = {
    'k_tx_1': "vo",
    'k_tx_2': "vo",
    'k_deg_1': "vo",
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=300, settings=settings)

k_tx_1 lower bound: 0.0, status: OPTIMAL, time: 27.896000146865845
k_tx_1 upper bound: 100.0, status: OPTIMAL, time: 69.05900001525879
k_tx_2 lower bound: 1.999940230403743, status: OPTIMAL, time: 295.2920000553131
k_tx_2 upper bound: 2.0034895504986414, status: TIME_LIMIT, time: 300.0329999923706
k_deg_1 lower bound: 0.0, status: OPTIMAL, time: 24.875
k_deg_1 upper bound: 50.000740740794846, status: OPTIMAL, time: 37.91600012779236
k_deg_2 = 1
k_reg lower bound: 0.0, status: OPTIMAL, time: 34.68400001525879
k_reg upper bound: 0.0007958884171124418, status: TIME_LIMIT, time: 300.10000014305115


: 

In [ ]:
# parameter settings
params_optim = {
    'k_tx_1': "v",
    'k_tx_2': "v",
    'k_deg_1': "v",
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=600, settings=settings)

k_tx_1: variable
k_tx_2: variable
k_deg_1: variable
k_deg_2 = 1
k_reg lower bound: 0.0, status: OPTIMAL, time: 15.33299994468689
k_reg upper bound: 0.0008841186750534124, status: TIME_LIMIT, time: 600.0280001163483


### One degradation rate fixed + no joint B constraints

In [68]:
# parameter settings
params_optim = {
    'k_tx_1': "vo",
    'k_tx_2': "vo",
    'k_deg_1': "vo",
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': False,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=300, settings=settings)

k_tx_1 lower bound: 0.0, status: OPTIMAL, time: 0.06700015068054199
k_tx_1 upper bound: 100.0, status: OPTIMAL, time: 0.002000093460083008
k_tx_2 lower bound: 0.0, status: OPTIMAL, time: 0.013000011444091797
k_tx_2 upper bound: 100.0, status: OPTIMAL, time: 0.01699995994567871
k_deg_1 lower bound: 0.0, status: OPTIMAL, time: 0.0
k_deg_1 upper bound: 100.0, status: OPTIMAL, time: 0.016000032424926758
k_deg_2 = 1
k_reg lower bound: 0.0, status: OPTIMAL, time: 0.01699995994567871
k_reg upper bound: 100.0, status: OPTIMAL, time: 0.0


# Testing with interaction

We would like to test if perfect information produces perfect results in the setting where $k_{reg} > 0$, but there is no analytical stationary solution available when interaction is present (to my knowledge).

Instead we use an extremely large sample and bootstrap size to produce the tightest possible confidence intervals, hopefully close to the exact vlues.

If the results of optimization are also tight bounds on the true parameter values then this suggests the same conclusions drawn above apply, and the optimization constraints are strong enough.

## Code: bounds computation

In [1]:
def simulation(params, n, beta=1.0, tmax=100, ts=10, plot=False, initial_state=(0, 0)):
    '''
    Simulate a sample path of the birth death regulation model
    Sample n values at intervals of ts after a burn-in time of tmax

    params: dict of reaction rate constants
    n: number of samples
    beta: capture efficiency, list of length n (per cell) or single value
    tmax: burn in time
    ts: time between samples
    '''

    # initialise time and state
    t = 0
    path = [initial_state]
    jump_times = [0]

    # simulate for burn-in time and time between n samples
    while t < tmax + (n - 1) * ts:

        # current state
        x1, x2 = path[-1][0], path[-1][1]

        # transition rates
        q_tx_1 = params['k_tx_1']
        q_tx_2 = params['k_tx_2']
        q_deg_1 = x1 * params['k_deg_1']
        q_deg_2 = x2 * params['k_deg_2']
        q_reg = x1 * x2 * params['k_reg']
        q_hold = q_tx_1 + q_tx_2 + q_deg_1 + q_deg_2 + q_reg

        # holding time in current state
        t_hold = -np.log(rng.uniform()) / q_hold
        t += t_hold
        jump_times.append(t)

        # jump probability
        outcome = [1, 2, 3, 4, 5]
        prob = [
            q_tx_1 / q_hold,
            q_tx_2 / q_hold,
            q_deg_1 / q_hold,
            q_deg_2 / q_hold,
            q_reg / q_hold
        ]
        jump = rng.choice(outcome, p=prob)
        match jump:
            case 1:
                path.append((x1 + 1, x2))
            case 2:
                path.append((x1, x2 + 1))
            case 3:
                path.append((x1 - 1, x2))
            case 4:
                path.append((x1, x2 - 1))
            case 5:
                path.append((x1 - 1, x2 - 1))

    # take the transcript states
    x1_path = [state[0] for state in path]
    x2_path = [state[1] for state in path]

    # create step function of sample path from jump times and jump values
    x1_path_function = scipy.interpolate.interp1d(jump_times, x1_path, kind='previous')
    x2_path_function = scipy.interpolate.interp1d(jump_times, x2_path, kind='previous')

    # take values at sampling times as samples from stationary dist
    sample_times = [tmax + i * ts for i in range(n)]
    x1_samples = x1_path_function(sample_times)
    x2_samples = x2_path_function(sample_times)

    # convert to integers
    x1_samples = [int(x1) for x1 in x1_samples]
    x2_samples = [int(x2) for x2 in x2_samples]

    # apply capture efficiency: for each count, draw from Binomial(count, beta)
    x1_samples_beta = np.random.binomial(x1_samples, beta).tolist()
    x2_samples_beta = np.random.binomial(x2_samples, beta).tolist()

    # re-combine to pairs of samples
    samples = list(zip(x1_samples, x2_samples))
    samples_beta = list(zip(x1_samples_beta, x2_samples_beta))

    # plot sample paths
    if plot:
        x = np.linspace(0, tmax + (n - 1) * ts, 10000)
        plt.plot(x, x1_path_function(x), label="X1 sample path", color="blue")
        plt.plot(x, x2_path_function(x), label="X2 sample path", color="purple")
        #plt.axvline(tmax, label="Burn-in time", color="orange")
        plt.xlabel("Time")
        plt.ylabel("# molcules")
        plt.legend()
        plt.show()

    # collect all sample paths: original and observed
    data = {
        'x1_OG': x1_samples,
        'x2_OG': x2_samples,
        'OG': samples,
        'x1_OB': x1_samples_beta,
        'x2_OB': x2_samples_beta,
        'OB': samples_beta
    }

    return data

In [23]:
def bootstrap_bounds(params, n, BS=1000, splits=1, beta=1.0, thresh_OB=10, tmax=100, ts=10, plot=False, printing=False):

    # simulate data
    data = simulation(params, n, beta, tmax, ts, plot=False, initial_state=(0, 0))

    # extract obserbed data
    samples = data['OB']

    # compute maximum x1 and x2 values
    M, N = np.max(samples, axis=0)
    M, N = int(M), int(N)

    # map (x1, x2) pairs to integers: x2 + (N + 1) * x1
    integer_samples = np.array([x[1] + (N + 1)*x[0] for x in samples], dtype='uint32')

    # maxiumum of integer sample
    D = (M + 1)*(N + 1) - 1

    # number of bootstrap samples per split
    BS_split = BS // splits

    # setup count array
    counts = np.empty((BS, M + 1, N + 1), dtype='uint32')

    # BS bootstrap samples: split into 'splits' number of BS_split x n arrays
    for split in tqdm.tqdm(range(splits)):

        # BS_split bootstrap samples as BS_split x n array
        bootstrap_split = rng.choice(integer_samples, size=(BS_split, n))

        # offset row i by (D + 1)i
        bootstrap_split += np.arange(BS_split, dtype='uint32')[:, None]*(D + 1)

        # flatten, count occurances of each state and reshape, reversing map to give counts of each (x1, x2) pair
        counts_split = np.bincount(bootstrap_split.ravel(), minlength=BS_split*(D + 1)).reshape(-1, M + 1, N + 1)

        # add to counts
        counts[(split * BS_split):((split + 1) * BS_split), :, :] = counts_split

    # sum over columns / rows to give counts (/n) of each x1 / x2 state
    x1_counts = counts.sum(axis=2)
    x2_counts = counts.sum(axis=1)

    # compute 2.5% and 97.5% quantiles for each p(x1, x2), p(x1) and p(x2)
    bounds = np.quantile(counts, [0.025, 0.975], axis=0)
    x1_bounds = np.quantile(x1_counts, [0.025, 0.975], axis=0)
    x2_bounds = np.quantile(x2_counts, [0.025, 0.975], axis=0)

    # scale to probability
    bounds = bounds / n
    x1_bounds = x1_bounds / n
    x2_bounds = x2_bounds / n

    # count occurances per (x1, x2) in the in original sample
    sample_counts = np.bincount(integer_samples, minlength=D + 1).reshape(M + 1, N + 1)

    # sum over columns / rows to give counts per x1 / x2 state
    x1_sample_counts = sample_counts.sum(axis=1)
    x2_sample_counts = sample_counts.sum(axis=0)

    # set truncation bounds
    min_x1_OB, max_x1_OB, min_x2_OB, max_x2_OB = M, 0, N, 0
    minM_x1_OB, maxM_x1_OB = M, 0
    minM_x2_OB, maxM_x2_OB = N, 0

    # set flag for changes
    thresh_flag = False
    thresh_flag_x1 = False
    thresh_flag_x2 = False

    # replace CI's for states below threshold occurances by [0, 1] bounds
    for x1 in range(M + 1):
        for x2 in range(N + 1):
            # below: replace
            if sample_counts[x1, x2] < thresh_OB:
                bounds[:, x1, x2] = [0.0, 1.0]
            # above: update truncation
            else:
                # check if smaller than current min
                if x1 < min_x1_OB:
                    min_x1_OB = x1
                    thresh_flag = True
                if x2 < min_x2_OB:
                    min_x2_OB = x2
                    thresh_flag = True
                # check if larger than current max
                if x1 > max_x1_OB:
                    max_x1_OB = x1
                    thresh_flag = True
                if x2 > max_x2_OB:
                    max_x2_OB = x2
                    thresh_flag = True

    for x1 in range(M + 1):
        # below: replace
        if x1_sample_counts[x1] < thresh_OB:
            x1_bounds[:, x1] = [0.0, 1.0]
        # above: update truncation
        else:
            # check if smaller than current min
            if x1 < minM_x1_OB:
                minM_x1_OB = x1
                thresh_flag_x1 = True
            # check if larger than current max
            if x1 > maxM_x1_OB:
                maxM_x1_OB = x1
                thresh_flag_x1 = True

    for x2 in range(N + 1):
        # below: replace
        if x2_sample_counts[x2] < thresh_OB:
            x2_bounds[:, x2] = [0.0, 1.0]
        # above: update truncation
        else:
            # check if smaller than current min
            if x2 < minM_x2_OB:
                minM_x2_OB = x2
                thresh_flag_x2 = True
            # check if larger than current max
            if x2 > maxM_x2_OB:
                maxM_x2_OB = x2
                thresh_flag_x2 = True

    # if no states were above threshold: default to max range, report
    if not thresh_flag:
        min_x1_OB, max_x1_OB, min_x2_OB, max_x2_OB = 0, M, 0, N
    if not thresh_flag_x1:
        minM_x1_OB, maxM_x1_OB = 0, M
    if not thresh_flag_x2:
        minM_x2_OB, maxM_x2_OB = 0, N

    # plotting
    if plot:
        fig, axs = plt.subplots(M + 1, N + 1, figsize=(10, 10))
        fig.tight_layout()
        for x1 in range(M + 1):
            for x2 in range(N + 1):
                # within truncation: green CI lines
                if (x1 >= min_x1_OB) and (x2 >= min_x2_OB) and (x1 <= max_x1_OB) and (x2 <= max_x2_OB):
                    color = "green"
                else:
                    color = "red"
                axs[x1, x2].hist(counts[:, x1, x2] / n)
                axs[x1, x2].set_title(f"p({x1}, {x2})")
                axs[x1, x2].axvline(bounds[0, x1, x2], color=color)
                axs[x1, x2].axvline(bounds[1, x1, x2], color=color)

        plt.suptitle("X1 X2 Confidence Intervals")
        plt.show()

        fig, axs = plt.subplots(1, M + 1, figsize=(10, 3))
        fig.tight_layout()
        for x1 in range(M + 1):
            # within truncation: green CI lines
            if (x1 >= minM_x1_OB) and (x1 <= maxM_x1_OB):
                color = "green"
            else:
                color = "red"
            axs[x1].hist(x1_counts[:, x1] / n)
            axs[x1].set_title(f"p({x1})")
            axs[x1].axvline(x1_bounds[0, x1], color=color)
            axs[x1].axvline(x1_bounds[1, x1], color=color)

        plt.suptitle("X1 Confidence Intervals")
        plt.show()

        fig, axs = plt.subplots(1, N + 1, figsize=(10, 3))
        fig.tight_layout()
        for x2 in range(N + 1):
            # within truncation: green CI lines
            if (x2 >= minM_x2_OB) and (x2 <= maxM_x2_OB):
                color = "green"
            else:
                color = "red"
            axs[x2].hist(x2_counts[:, x2] / n)
            axs[x2].set_title(f"p({x2})")
            axs[x2].axvline(x2_bounds[0, x2], color=color)
            axs[x2].axvline(x2_bounds[1, x2], color=color)

        plt.suptitle("X2 Confidence Intervals")
        plt.show()

    if printing:
        print(f"Box truncation: [{min_x1_OB}, {max_x1_OB}] x [{min_x2_OB}, {max_x2_OB}]")
        print(f"Marginal x1 truncation: [{minM_x1_OB}, {maxM_x1_OB}]")
        print(f"Marginal x2 truncation: [{minM_x2_OB}, {maxM_x2_OB}]")

    results =  {
        'samples': samples,
        'sample_counts': sample_counts,
        'sample_counts_x1': x1_sample_counts,
        'sample_counts_x2': x2_sample_counts,
        'joint': bounds,
        'x1': x1_bounds,
        'x2': x2_bounds,
        'min_x1_OB': min_x1_OB,
        'max_x1_OB': max_x1_OB,
        'min_x2_OB': min_x2_OB,
        'max_x2_OB': max_x2_OB,
        'minM_x1_OB': minM_x1_OB,
        'maxM_x1_OB': maxM_x1_OB,
        'minM_x2_OB': minM_x2_OB,
        'maxM_x2_OB': maxM_x2_OB,
        'thresh_flag': thresh_flag,
        'thresh_flag_x1': thresh_flag_x1,
        'thresh_flag_x2': thresh_flag_x2
    }

    return results

## Test: (hyp) marginal optimization, (almost) perfect information

In [27]:
# reaction rates: no interaction
params = {
    'k_tx_1': 2,
    'k_tx_2': 2,
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': 2
}

# capture efficiency
beta = 0.5

# truncations
truncationsM = {}

# compute almost exact bounds
bounds = bootstrap_bounds(params, n=10000, BS=10000, splits=1, beta=beta, thresh_OB=10, tmax=100, ts=10, plot=False, printing=True)

100%|██████████| 1/1 [00:05<00:00,  5.34s/it]


Box truncation: [0, 4] x [0, 4]
Marginal x1 truncation: [0, 4]
Marginal x2 truncation: [0, 4]


In [28]:
solution_x1 = optimization_hyp_single(bounds, beta, truncationsM, gene=1, K=100, silent=True,
                     print_solution=True, print_truncation=False, threshM_OG=10**-6,
                     time_limit=300)

k_tx lower bound: None, status: INFEASIBLE, time: 0.07800006866455078
k_tx upper bound: None, status: INFEASIBLE, time: 0.017999887466430664
k_deg: 1


In [29]:
solution_x2 = optimization_hyp_single(bounds, beta, truncationsM, gene=2, K=100, silent=True,
                     print_solution=True, print_truncation=False, threshM_OG=10**-6,
                     time_limit=300)

k_tx lower bound: None, status: INFEASIBLE, time: 0.011999845504760742
k_tx upper bound: None, status: INFEASIBLE, time: 0.012000083923339844
k_deg: 1


Infeasible (only need marginal bounds), expected as hyp assumption is of no interaction

## Test: (hyp) full optimization, (almost) perfect information

In [31]:
# reaction rates: no interaction
params = {
    'k_tx_1': 2,
    'k_tx_2': 2,
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': 2
}

# capture efficiency
beta = 0.5

# truncations (will be trivial i.e. x_OB: (x_OB, x_OB) as 100% capture)
truncations = {}
truncationsM = {}

# compute almost exact bounds
bounds = bootstrap_bounds(params, n=10000, BS=10000, splits=1, beta=beta, thresh_OB=10, tmax=100, ts=10, plot=False, printing=True)

100%|██████████| 1/1 [00:02<00:00,  2.40s/it]

Box truncation: [0, 4] x [0, 4]
Marginal x1 truncation: [0, 4]
Marginal x2 truncation: [0, 4]


In [32]:
# parameter settings
params_optim = {
    'k_tx_1': "vo",
    'k_tx_2': "vo",
    'k_deg_1': 1,
    'k_deg_2': 1,
}

# constraint settings
settings = {
    'bivariateB': False,
    'univariateB': True,
    'bivariateCME': False,
    'univariateCME': True
}

# optimize
solution = optimization_hyp(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=300, settings=settings)

k_tx_1 lower bound: None, status: INFEASIBLE, time: 0.057000160217285156
k_tx_1 upper bound: None, status: INFEASIBLE, time: 0.04699993133544922
k_tx_2 lower bound: None, status: INFEASIBLE, time: 0.04999995231628418
k_tx_2 upper bound: None, status: INFEASIBLE, time: 0.046000003814697266
k_deg_1 = 1
k_deg_2 = 1


Infeasible (only need marginal bounds), expected as hyp assumption is of no interaction

## Test: (min) standard optimization, (almost) perfect information

In [36]:
# reaction rates: no interaction
params = {
    'k_tx_1': 2,
    'k_tx_2': 2,
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': 2
}

# capture efficiency
beta = 0.5

# truncations (will be trivial i.e. x_OB: (x_OB, x_OB) as 100% capture)
truncations = {}
truncationsM = {}

# compute almost exact bounds
bounds = bootstrap_bounds(params, n=100000, BS=100000, splits=100, beta=beta, thresh_OB=10, tmax=100, ts=10, plot=False, printing=True)

100%|██████████| 100/100 [02:57<00:00,  1.77s/it]


Box truncation: [0, 5] x [0, 5]
Marginal x1 truncation: [0, 5]
Marginal x2 truncation: [0, 5]


### Both degradation rates fixed

In [54]:
# parameter settings
params_optim = {
    'k_tx_1': "v",
    'k_tx_2': "v",
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=False,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=600, settings=settings)

Set parameter TimeLimit to value 600
Set parameter Presolve to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 221 rows, 1685 columns and 36818 nonzeros
Model fingerprint: 0x0c4708e2
Model has 1521 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [4e-05, 1e+00]
Presolve removed 89 rows and 2 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 134 rows and 2 columns
Presolve time: 0.04s
Presolved: 19860 rows, 6267 columns, 60422 nonzeros
Presolved model has 4563 bilinear constraint(s)
Variable types: 6267 continuous, 0 integer (0 binary)
Root relaxation presolve removed 4578 rows a

#### Time extention

In [64]:
# parameter settings
params_optim = {
    'k_tx_1': "v",
    'k_tx_2': "v",
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=False,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=5400, settings=settings)

Set parameter TimeLimit to value 5400
Set parameter Presolve to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 221 rows, 1685 columns and 36818 nonzeros
Model fingerprint: 0x0c4708e2
Model has 1521 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [4e-05, 1e+00]
Presolve removed 89 rows and 2 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 134 rows and 2 columns
Presolve time: 0.03s
Presolved: 19860 rows, 6267 columns, 60422 nonzeros
Presolved model has 4563 bilinear constraint(s)
Variable types: 6267 continuous, 0 integer (0 binary)
Root relaxation presolve removed 4578 rows 

### Both degradation rates fixed + no joint B constraints

In [55]:
# parameter settings
params_optim = {
    'k_tx_1': "v",
    'k_tx_2': "v",
    'k_deg_1': 1,
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': False,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=600, settings=settings)

k_tx_1: variable
k_tx_2: variable
k_deg_1 = 1
k_deg_2 = 1
k_reg lower bound: 0.0, status: OPTIMAL, time: 1.0339999198913574
k_reg upper bound: 100.0, status: OPTIMAL, time: 0.06200003623962402


### One degradation rate fixed

In [58]:
# parameter settings
params_optim = {
    'k_tx_1': "v",
    'k_tx_2': "v",
    'k_deg_1': "v",
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=False,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=600, settings=settings)

Set parameter TimeLimit to value 600
Set parameter Presolve to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 220 rows, 1685 columns and 36817 nonzeros
Model fingerprint: 0x51449b94
Model has 1521 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [4e-05, 1e+00]
Presolve removed 88 rows and 1 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 133 rows and 1 columns
Presolve time: 0.04s
Presolved: 25944 rows, 7789 columns, 74034 nonzeros
Presolved model has 6084 bilinear constraint(s)
Variable types: 7789 continuous, 0 integer (0 binary)
Root relaxation presolve removed 6099 rows a

#### Time extention

In [63]:
# parameter settings
params_optim = {
    'k_tx_1': "v",
    'k_tx_2': "v",
    'k_deg_1': "v",
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=False,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=5400, settings=settings)

Set parameter TimeLimit to value 5400
Set parameter Presolve to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 220 rows, 1685 columns and 36817 nonzeros
Model fingerprint: 0x51449b94
Model has 1521 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [4e-05, 1e+00]
Presolve removed 88 rows and 1 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 133 rows and 1 columns
Presolve time: 0.05s
Presolved: 25944 rows, 7789 columns, 74034 nonzeros
Presolved model has 6084 bilinear constraint(s)
Variable types: 7789 continuous, 0 integer (0 binary)
Root relaxation presolve removed 6099 rows 

#### Time extention

In [65]:
# parameter settings
params_optim = {
    'k_tx_1': "v",
    'k_tx_2': "v",
    'k_deg_1': "v",
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': True,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=False,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=10800, settings=settings)

Set parameter TimeLimit to value 10800
Set parameter Presolve to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 220 rows, 1685 columns and 36817 nonzeros
Model fingerprint: 0x51449b94
Model has 1521 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [4e-05, 1e+00]
Presolve removed 88 rows and 1 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 133 rows and 1 columns
Presolve time: 0.05s
Presolved: 25944 rows, 7789 columns, 74034 nonzeros
Presolved model has 6084 bilinear constraint(s)
Variable types: 7789 continuous, 0 integer (0 binary)
Root relaxation presolve removed 6099 rows

### One degradation rate fixed + no joint B constraints

In [60]:
# parameter settings
params_optim = {
    'k_tx_1': "v",
    'k_tx_2': "v",
    'k_deg_1': "v",
    'k_deg_2': 1,
    'k_reg': "vo"
}

# constraint settings
settings = {
    'bivariateB': False,
    'univariateB': True,
    'bivariateCME': True,
}

# optimize
solution = optimization_min(params_optim, bounds, beta, truncations, truncationsM, K=100, silent=True,
                     print_solution=True, print_truncation=False, thresh_OG=10**-6, threshM_OG=10**-6,
                     time_limit=300, settings=settings)

k_tx_1: variable
k_tx_2: variable
k_deg_1: variable
k_deg_2 = 1
k_reg lower bound: 0.0, status: OPTIMAL, time: 0.934999942779541
k_reg upper bound: 100.0, status: OPTIMAL, time: 0.09999990463256836
